# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [7]:
import requests
import pandas as pd

# Define Foursquare function
def get_foursquare_data(lat, lon, api_key):
    headers = {'Authorization': api_key}
    params = {'ll': f'{lat},{lon}', 'radius': 1000, 'query': 'restaurant'}
    response = requests.get('https://api.foursquare.com/v3/places/search', headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Foursquare API Error: {response.status_code}, {response.text}")
        return None

# Foursquare API key 
foursquare_api_key = 'fsq3GyfbX6HyEid5W13kCSuzPRsd0YmBh0Ipisz8epSVJ14='

# bike station data for New York City
bike_stations = pd.DataFrame({
    'Name': ['Station 1', 'Station 2', 'Station 3'],
    'Latitude': [40.748817, 40.761596, 40.764817],
    'Longitude': [-73.985428, -73.981618, -73.978508]
})

# Initialize result storage
foursquare_results = []

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
# Query Foursquare API for each bike station
for index, row in bike_stations.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    fs_data = get_foursquare_data(lat, lon, foursquare_api_key)
    if fs_data:
        for poi in fs_data.get('results', []):
            foursquare_results.append({
                'Station Name': row['Name'],
                'POI Name': poi.get('name', 'Unknown'),
                'Category': poi.get('categories', [{}])[0].get('name', 'Unknown'),
                'Latitude': poi.get('geocodes', {}).get('main', {}).get('latitude', None),
                'Longitude': poi.get('geocodes', {}).get('main', {}).get('longitude', None)
            })



Put your parsed results into a DataFrame

In [9]:
# Create a DataFrame for Foursquare results
foursquare_df = pd.DataFrame(foursquare_results)

# Save to CSV
foursquare_df.to_csv('foursquare_results_nyc.csv', index=False)

# Display the DataFrame
print("Foursquare Results DataFrame:")
print(foursquare_df.head())

Foursquare Results DataFrame:
  Station Name             POI Name           Category   Latitude  Longitude
0    Station 1            Rib No. 7          BBQ Joint  40.748329 -73.986845
1    Station 1  Yoon Haeundae Galbi          BBQ Joint  40.749849 -73.984502
2    Station 1     Tonchin New York                Bar  40.750335 -73.984477
3    Station 1       BCD Tofu House  Korean Restaurant  40.747437 -73.985820
4    Station 1     The Parlour Room       Cocktail Bar  40.750667 -73.986194


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [10]:
import requests
import pandas as pd

# Function to query Yelp API
def get_yelp_data(lat, lon, api_key, radius=1000, categories='restaurants,bars'):
    headers = {'Authorization': f'Bearer {api_key}'}
    params = {
        'latitude': lat,
        'longitude': lon,
        'categories': categories,
        'radius': radius
    }
    response = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Yelp API Error: {response.status_code}, {response.text}")
        return None

# Yelp API key
yelp_api_key = 'NXlcmooqZ6zbzXVDV8aFaMceh8PUtCS3YjJfSG0XUNOSNJA2xHL4ovwJ7VH0qTvDjsF_xOC2cP_kQ0Xxuv-5UhcCx5TYJ_LqVGPYS7VVX2iZg9MC3BIFhzuPTRX4Z3Yx'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [11]:
# bike station data for New York City
bike_stations = pd.DataFrame({
    'Name': ['Station 1', 'Station 2', 'Station 3'],
    'Latitude': [40.748817, 40.761596, 40.764817],
    'Longitude': [-73.985428, -73.981618, -73.978508]
})

# Initialize a list to store the parsed POI details
yelp_results = []

# parse Yelp API for each bike station
for index, row in bike_stations.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    response_data = get_yelp_data(lat, lon, yelp_api_key, radius=1000, categories='restaurants,bars')
    
    if response_data:
        for business in response_data.get('businesses', []):
            yelp_results.append({
                'Station Name': row['Name'],
                'POI Name': business.get('name', 'Unknown'),
                'Rating': business.get('rating', 'N/A'),
                'Review Count': business.get('review_count', 'N/A'),
                'Latitude': business.get('coordinates', {}).get('latitude', None),
                'Longitude': business.get('coordinates', {}).get('longitude', None),
                'Address': business.get('location', {}).get('address1', 'N/A'),
                'Distance (m)': business.get('distance', 'N/A')
            })

Put your parsed results into a DataFrame

In [13]:
# Create a DataFrame from the Yelp results
yelp_df = pd.DataFrame(yelp_results)

# Save the DataFrame to a CSV file
yelp_df.to_csv('yelp_poi_data_nyc.csv', index=False)

# Display the DataFrame
print("Yelp POI DataFrame:")
print(yelp_df.head())

Yelp POI DataFrame:
  Station Name                                           POI Name  Rating  \
0    Station 1                                            Tonchin     4.4   
1    Station 1                                        Izakaya MEW     4.3   
2    Station 1                                   Anytime Kitchen      4.6   
3    Station 1  The Ragtrader & Bo Peep Cocktail and Highball ...     4.3   
4    Station 1                                        Barn Joo 35     4.2   

   Review Count   Latitude  Longitude          Address  Distance (m)  
0          2345  40.750273 -73.984495     13 W 36th St    179.934357  
1          3166  40.750148 -73.986320     53 W 35th St    165.995569  
2          3169  40.747763 -73.986779     23 W 32nd St    163.408392  
3           993  40.750711 -73.986219  70 W 36th St 1A    220.872893  
4          2524  40.749640 -73.985820     34 W 35th St     99.812830  


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp has provided better coverage in terms of quantity of POI, It has more information on rating and reviews and offer detailed addresses. 

Get the top 10 restaurants according to their rating

Anytime Kitchen 
CHILI
Nan Xiang Soup Dumplings - Koreatown
Tuscany Steakhouse
Nubiani
Zen Ramen & Sushi
Haswell Green's
Bartley Dunnes
Bengal Tiger 
Beyond Sushi